# Spleeter Separation

In [ ]:
import os
import time
import keras
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from spleeter.audio import Codec, STFTBackend
from spleeter.audio.adapter import AudioAdapter
from spleeter.separator import Separator
from IPython.display import display, Audio

### Separation Function

In [ ]:
def separate(in_file, model):
    """
    Run Spleeter source separation model on WAV files to isolate stems.

    :param in_file: (str) path of mixture WAV file to separate
    :param model: (spleeter.separator.Separator) spleeter model object
    """  
    # only process wav files
    if in_file.endswith(".wav"):
        # read the soundfile
        waveform, _ = ADAPTER.load(in_file, sample_rate=SAMPLE_RATE)
    
        out = model.separate(waveform)
        out_stems = []
        for s in STEMS:
            est_stem = out[s]
            out_stems.append(est_stem)

    else:
        out_stems = None
        print("Invalid input file type. Please use WAV files only")

    return out_stems

### Separate

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]
SAMPLE_RATE = 44100

In [ ]:
# set up audio adapter
adapter_type = "spleeter.audio.ffmpeg.FFMPEGProcessAudioAdapter"
ADAPTER = AudioAdapter.get(adapter_type)
BITRATE = "128k"
CODEC = Codec.WAV

#### Stereo Mixtures

In [ ]:
# set input and output directories
INPUT_DIR = "../data/musdb18hq/test/"
OUTPUT_DIR = "../data/output/spleeter/stereo/test/"

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
file_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
print(f"There are {len(file_list)} files in the input directory.")

In [ ]:
# create the output directory if it does not already exist
print("Creating output directory, if it does not already exist...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# iterate through each file
print("Beginning to process files...")
for file in tqdm(file_list):
    # load the model in each loop because of a bug in spleeter 2.3.2
    sess = tf.compat.v1.InteractiveSession()
    MODEL = Separator('spleeter:4stems', MWF=False, stft_backend=STFTBackend.AUTO, multiprocess=False)
    out_dir = os.path.join(OUTPUT_DIR, file)
    os.makedirs(out_dir, exist_ok=True)
    out = separate(os.path.join(INPUT_DIR, file, 'mixture.wav'), MODEL)
    for i in range(len(STEMS)):
        out_path = os.path.join(out_dir, STEMS[i] + '.wav')
        ADAPTER.save(out_path, out[i], SAMPLE_RATE, CODEC, BITRATE)
    sess.close()
    time.sleep(5)  # addresses race condition issue
print("Processing complete!")

#### Binaural Mixtures

In [ ]:
# set input and output directories
INPUT_DIR = "../data/binaural_musdb18/test/"
OUTPUT_DIR = "../data/output/spleeter/binaural/test/"

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
file_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
print(f"There are {len(file_list)} files in the input directory.")

In [ ]:
# create the output directory if it does not already exist
print("Creating output directory, if it does not already exist...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# iterate through each file
print("Beginning to process files...")
for file in tqdm(file_list):
    # load the model in each loop because of a bug in spleeter 2.3.2
    sess = tf.compat.v1.InteractiveSession()
    MODEL = Separator('spleeter:4stems', MWF=False, stft_backend=STFTBackend.AUTO, multiprocess=False)
    out_dir = os.path.join(OUTPUT_DIR, file)
    os.makedirs(out_dir, exist_ok=True)
    out = separate(os.path.join(INPUT_DIR, file, 'mixture.wav'), MODEL)
    for i in range(len(STEMS)):
        out_path = os.path.join(out_dir, STEMS[i] + '.wav')
        ADAPTER.save(out_path, out[i], SAMPLE_RATE, CODEC, BITRATE)
    sess.close()
    time.sleep(5)  # addresses race condition issue
print("Processing complete!")